## BERT
#### Since this method requres time. I could not test it with big data
#### the code I show below could run on the same dataset I used for BoW method. The only thing needed is training 
#### And Prof you could see from the print out, which shows that only 96 data volume require 10 min for one epoch. 


In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
from torch.optim import Adam
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd



In [5]:
data_news = pd.read_csv("AAPL.csv", error_bad_lines=False,delimiter = "\t")
data_price = pd.read_csv('AAPL_price.csv')
data_price = data_price[['Date','Adj Close']].copy()
data_price['lead'] = data_price['Adj Close'].shift(-1)
data_price['lag'] = data_price['Adj Close'].shift(1)
data_price['label'] = data_price['lead'] > data_price['lag']
data_price['label'] = data_price['label'].astype(int)

In [6]:
data_price = data_price[['Date','label']]

In [7]:
data_price['Date'] = pd.to_datetime(data_price['Date'])
data_news['date'] = pd.to_datetime(data_news['date'])

In [8]:
data = pd.merge(data_news,
                data_price[['Date','label']].rename(columns = {'Date':'date'}),
                on = 'date',
                how = 'left'
                
                )

In [13]:
data

,date,time,ticker,name,title,summary,link,label
0,2021-12-19,2021-12-19 04:34:20,AAPL,Apple Inc.,Exclusive-Apple seeks dismissal of India apps ...,Apple Inc has asked India's antitrust watchdog...,https://finance.yahoo.com/news/exclusive-apple...,NaN
1,2021-12-19,2021-12-19 01:25:39,AAPL,Apple Inc.,Dow Jones Futures: Why This Market Rally Is So...,The market rally is chopping up investors fina...,https://finance.yahoo.com/m/b19de6ca-ffa2-3872...,NaN
2,2021-12-18,2021-12-18 16:15:00,AAPL,Apple Inc.,3 Top Energy Stocks Ready for a Bull Run,Decarbonization might be one of the biggest me...,https://finance.yahoo.com/m/1edd438b-e080-3f5e...,NaN
3,2021-12-18,2021-12-18 14:22:14,AAPL,Apple Inc.,Why Bruce Springsteen's $500M deal signals a '...,"""The Boss"" is cashing in — and that could have...",https://finance.yahoo.com/news/why-bruce-sprin...,NaN
4,2021-12-18,2021-12-18 12:05:00,AAPL,Apple Inc.,3 Stocks That Could Be Worth More Than Apple b...,Apple (NASDAQ: AAPL) is currently the world's ...,https://finance.yahoo.com/m/d3ac17b8-2954-37c8...,NaN
...,...,...,...,...,...,...,...,...
58684,2018-01-01,2018-01-01 23:28:00,AAPL,Apple Inc.,"Forget Your iPhone X, Ignore The Samsung And P...","Looking back on 2017, the smartphone industry ...",http://finance.yahoo.com/r/2e9f187e-56ab-319e-...,NaN
58685,2018-01-01,2018-01-01 22:19:00,AAPL,Apple Inc.,Apple's Executive Cash Bonus Plan,Apple’s executives have done very well over th...,http://finance.yahoo.com/r/a212bfe1-4a00-3d00-...,NaN
58686,2018-01-01,2018-01-01 21:08:20,AAPL,Apple Inc.,2 Warren Buffett Stocks to Consider Buying Now,A 1 000 investment in Berkshire Hathaway 160 s...,http://articlefeeds.nasdaq.com/~r/nasdaq/symbo...,NaN
58687,2018-01-01,2018-01-01 20:02:00,AAPL,Apple Inc.,2 Warren Buffett Stocks to Consider Buying Now,The legendary investor knows a thing or two ab...,https://finance.yahoo.com/news/2-warren-buffet...,NaN


In [6]:
data_trail = data.dropna().head(160)

In [7]:
model_name = 'bert-base-uncased'


In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
# example_text = 'I will watch Memento tonight'
# bert_input = tokenizer(example_text,padding='max_length', 
#                        max_length = 512, 
#                        truncation=True,
#                        return_tensors="pt")
#print(bert_input['input_ids'])
# print(bert_input['token_type_ids'])
# print(bert_input['attention_mask'])

In [9]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,data_trail ):
        self.labels = [data_trail['label'].iloc[i] for i in range(len(data_trail))]
        self.texts = [tokenizer(data_trail['title'].iloc[i], 
                                padding='max_length', 
                                max_length = 512, 
                                truncation=True,
                                return_tensors="pt") 
                      for i in range(len(data_trail))]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [10]:
df_train, df_val, df_test = np.split(data_trail.sample(frac=1, random_state=42), 
                                     [int(.6*len(data_trail)), int(.8*len(data_trail))])

print(len(df_train),len(df_val), len(df_test))

96 32 32


In [11]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.linear = nn.Linear(768, 1)
        self.Sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        linear_output = self.linear(pooled_output)
        final_layer = self.Sigmoid(linear_output)
        return final_layer

In [12]:
train_data = df_train
val_data = df_val
epochs = 5
model = BertClassifier()
learning_rate = 1e-6

In [14]:


train, val = Dataset(train_data), Dataset(val_data)

train_dataloader = torch.utils.data.DataLoader(train, batch_size=40, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=40)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=learning_rate)

if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            output = output.flatten()
            #train_label = train_label.long()
            #output = output.double()
            train_label = train_label.double()
            output = output.double()

            batch_loss = criterion(output, train_label)
            total_loss_train += batch_loss.item()

            acc = (output == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                output = output.flatten()
                #train_label = train_label.long()
                #output = output.double()
                val_label = val_label.double()
                output = output.double()
                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()
                
                acc = (output == val_label).sum().item()
                total_acc_val += acc
        
        print(
            f'''Epochs: {epoch_num + 1} 
            | Train Loss: {total_loss_train / len(train_data): .3f} 
            | Train Accuracy: {total_acc_train / len(train_data): .3f} 
            | Val Loss: {total_loss_val / len(val_data): .3f} 
            | Val Accuracy: {total_acc_val / len(val_data): .3f}''')  

  0%|          | 0/3 [00:00<?, ?it/s]

Epochs: 1 
            | Train Loss:  0.020 
            | Train Accuracy:  0.000 
            | Val Loss:  0.019 
            | Val Accuracy:  0.000


  0%|          | 0/3 [00:00<?, ?it/s]

Epochs: 2 
            | Train Loss:  0.020 
            | Train Accuracy:  0.000 
            | Val Loss:  0.019 
            | Val Accuracy:  0.000


  0%|          | 0/3 [00:00<?, ?it/s]

Epochs: 3 
            | Train Loss:  0.020 
            | Train Accuracy:  0.000 
            | Val Loss:  0.019 
            | Val Accuracy:  0.000
